In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
import time
import os
print ("PACKAGES LOADED")

PACKAGES LOADED


In [2]:
A1 = np.loadtxt('data\Input_Altitude.txt')
A2 = np.loadtxt('data\Output_Altitude.txt')
A3 = np.loadtxt('data\Altitude.txt')

In [3]:
print(A1.shape)
print(A2.shape)
print(A3.shape)

(40000, 100)
(40000, 300)
(40000,)


In [4]:
mean_in = np.mean(A1, axis=0)
std_in = np.std(A1, axis=0)
B1 = (A1 - mean_in) / std_in
B2 = A2 

In [5]:
X_train, X_test, Y_train, Y_test, in_train, in_test, out_train, out_test, H_train, H_test = train_test_split(
   B1, B2, A1, A2, A3, test_size=0.10, random_state=0)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(H_train.shape)
print(H_test.shape)
num_examples=X_train.shape[0]

(36000, 100)
(36000, 300)
(4000, 100)
(4000, 300)
(36000,)
(4000,)


In [6]:
np.savetxt('network_save\mean_ATEM.txt', mean_in)
np.savetxt('network_save\std_ATEM.txt', std_in)
print('Ready!!')

In [7]:
print(H_test)

[48. 59. 58. ... 73. 70. 41.]


In [8]:
np.savetxt('data\input_altitude_test.txt', in_test)
np.savetxt('data\output_altitude_test.txt', out_test)
np.savetxt('data\height_test.txt',H_test)

In [9]:
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    a = tf.reduce_sum(tf.multiply(vector_a, vector_b), axis=1)
    b = tf.multiply(tf.norm(vector_a, axis=1), tf.norm(vector_b, axis=1))
    sim = tf.reduce_mean(tf.divide(a, b))
    return sim

In [10]:
# 实现Batch Normalization
def bn1_layer(x,is_training,name='BatchNorm',moving_decay=0.9,eps=1e-5):
    # 获取输入维度并判断是否匹配卷积层(4)或者全连接层(2)
    shape = x.shape
    assert len(shape) in [2,4]

    param_shape = shape[-1]
    with tf.variable_scope(name_or_scope='', reuse = tf.AUTO_REUSE):
#     with tf.variable_scope(name)：
        # 声明BN中唯一需要学习的两个参数，y=gamma*x+beta
        gamma = tf.get_variable('gamma1',param_shape,initializer=tf.constant_initializer(1))
        beta  = tf.get_variable('beat1', param_shape,initializer=tf.constant_initializer(0))

        # 计算当前整个batch的均值与方差
        axes = list(range(len(shape)-1))
        batch_mean, batch_var = tf.nn.moments(x,axes,name='moments')

        # 采用滑动平均更新均值与方差
        ema = tf.train.ExponentialMovingAverage(moving_decay)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean,batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        # 训练时，更新均值与方差，测试时使用之前最后一次保存的均值与方差
        mean, var = tf.cond(tf.equal(is_training,True),mean_var_with_update,
                lambda:(ema.average(batch_mean),ema.average(batch_var)))

        # 最后执行batch normalization
        return tf.nn.batch_normalization(x,mean,var,beta,gamma,eps)
    
    # 实现Batch Normalization
def bn2_layer(x,is_training,name='BatchNorm',moving_decay=0.9,eps=1e-5):
    # 获取输入维度并判断是否匹配卷积层(4)或者全连接层(2)
    shape = x.shape
    assert len(shape) in [2,4]

    param_shape = shape[-1]
    with tf.variable_scope(name_or_scope='', reuse = tf.AUTO_REUSE):
#     with tf.variable_scope(name)：
        # 声明BN中唯一需要学习的两个参数，y=gamma*x+beta
        gamma = tf.get_variable('gamma2',param_shape,initializer=tf.constant_initializer(1))
        beta  = tf.get_variable('beat2', param_shape,initializer=tf.constant_initializer(0))

        # 计算当前整个batch的均值与方差
        axes = list(range(len(shape)-1))
        batch_mean, batch_var = tf.nn.moments(x,axes,name='moments')

        # 采用滑动平均更新均值与方差
        ema = tf.train.ExponentialMovingAverage(moving_decay)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean,batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        # 训练时，更新均值与方差，测试时使用之前最后一次保存的均值与方差
        mean, var = tf.cond(tf.equal(is_training,True),mean_var_with_update,
                lambda:(ema.average(batch_mean),ema.average(batch_var)))

        # 最后执行batch normalization
        return tf.nn.batch_normalization(x,mean,var,beta,gamma,eps)
    
def bn3_layer(x,is_training,name='BatchNorm',moving_decay=0.9,eps=1e-5):
    # 获取输入维度并判断是否匹配卷积层(4)或者全连接层(2)
    shape = x.shape
    assert len(shape) in [2,4]

    param_shape = shape[-1]
    with tf.variable_scope(name_or_scope='', reuse = tf.AUTO_REUSE):
#     with tf.variable_scope(name)：
        # 声明BN中唯一需要学习的两个参数，y=gamma*x+beta
        gamma = tf.get_variable('gamma3',param_shape,initializer=tf.constant_initializer(1))
        beta  = tf.get_variable('beat3', param_shape,initializer=tf.constant_initializer(0))

        # 计算当前整个batch的均值与方差
        axes = list(range(len(shape)-1))
        batch_mean, batch_var = tf.nn.moments(x,axes,name='moments')

        # 采用滑动平均更新均值与方差
        ema = tf.train.ExponentialMovingAverage(moving_decay)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean,batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        # 训练时，更新均值与方差，测试时使用之前最后一次保存的均值与方差
        mean, var = tf.cond(tf.equal(is_training,True),mean_var_with_update,
                lambda:(ema.average(batch_mean),ema.average(batch_var)))

        # 最后执行batch normalization
        return tf.nn.batch_normalization(x,mean,var,beta,gamma,eps)

In [11]:
batch_size = 128
training_epochs = 1000
H_200_train = np.tile(H_train,(200,1))
H_200_train = np.transpose(H_200_train)
H_200_test = np.tile(H_test,(200,1))
H_200_test = np.transpose(H_200_test)
print(H_200_test.shape)
print(H_200_train.shape)

(4000, 200)
(36000, 200)


In [12]:
input_queue = tf.train.slice_input_producer([X_train, Y_train,H_200_train], shuffle=False)
xtrain_batch, ytrain_batch, htrain_batch = tf.train.shuffle_batch(input_queue,
                                                   batch_size=128, capacity=500,
                                                   min_after_dequeue=200, num_threads=2)

In [13]:
x = tf.placeholder(tf.float32, [None, 100])
y = tf.placeholder(tf.float32, [None, 300])
h = tf.placeholder(tf.float32, [None, 200])
keep_prob = tf.placeholder(tf.float32)
ornot = tf.placeholder(tf.float32)
x_image = tf.reshape(x, [-1, 1, 100, 1])  

In [14]:
def weights(shape):
    #给权重制造一些随机的噪声来打破完全对称，比如截断的正态分布噪声，标准差设为0.1 
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return w

def biases(shape):
    #给偏置增加一些小的正值(0.1)，来避免死亡节点(dead neurons) 
    b = tf.Variable(tf.constant(0.1,shape=shape))
    return b

def conv2d(x,W): 
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')  

def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   #池化尺寸[1,3,3,1]


In [15]:
# Conv1 Layer  
W_conv1 = weights([1, 15, 1, 32])  
b_conv1 = biases([32])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
# h_pool1 = bn2_layer(h_pool1,ornot)

# Conv2 Layer  
W_conv2 = weights([1, 15, 32, 64])  
b_conv2 = biases([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
# h_pool2 = bn3_layer(h_pool2,ornot)

# Conv2 Layer  
W_conv3 = weights([1, 5, 64, 128])  
b_conv3 = biases([128])  
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)  
h_pool3 = max_pool_2x2(h_conv3)  
# h_pool3 = bn4_layer(h_pool3,ornot)

#full connection  
W_fc1 = weights([13*128, 800])  
b_fc1 = biases([800])  
h_pool2_flat = tf.reshape(h_pool3, [-1, 13*128]) 
L1 = tf.add(tf.matmul(h_pool2_flat, W_fc1), b_fc1)
L1_bn = bn1_layer(L1,ornot)
h_fc1 = tf.nn.relu(L1_bn)
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

h_fc1_stack = tf.concat([h_fc1_drop, h],1)

W_fc2 = weights([1000, 600])  
b_fc2 = biases([600])  
L2 = tf.add(tf.matmul(h_fc1_stack, W_fc2), b_fc2)
L2_bn = bn2_layer(L2,ornot)
h_fc2 = tf.nn.relu(L2_bn)
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

w3 = tf.Variable(tf.truncated_normal([600, 300], stddev=0.1))
b3 = tf.Variable(tf.zeros([300]) + 0.1)
prediction = tf.matmul(h_fc2_drop,w3 ) + b3

In [16]:
saver = tf.train.Saver()

In [17]:
L2_reg = 0.001
regularizer = tf.contrib.layers.l2_regularizer(L2_reg)
reg_loss =regularizer(W_fc1) +  regularizer(w3)
#定义损失函数、优化器和准确率。
loss_sqr = tf.reduce_mean(tf.square(prediction - y))
loss_abs = tf.reduce_mean(tf.abs(prediction - y)) 
loss_cos= 1 - cos_sim(prediction , y)
#
loss = loss_sqr + reg_loss
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [18]:
start =time.clock() 
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    step = 0
    total_batch =  int(num_examples/batch_size)
    for epoch in range (training_epochs):
        for i in range (total_batch):
            batch_x, batch_y, batch_h=sess.run([xtrain_batch, ytrain_batch, htrain_batch])
            sess.run(train_step, feed_dict ={x:batch_x, y:batch_y, keep_prob:0.7,ornot:True, h:batch_h})
        if epoch % 1 ==0:
            loss_train = sess.run(loss, feed_dict ={x:batch_x, y:batch_y, keep_prob:1,ornot:False, h:batch_h})
            rmse_train = sess.run(loss_sqr, feed_dict ={x:batch_x, y:batch_y, keep_prob:1,ornot:False, h:batch_h})
            test_data = X_test
            test_label = Y_test
            rmse_test = sess.run(loss_sqr, feed_dict ={x:test_data, y:test_label, keep_prob:1.0,ornot:False, h:H_200_test})
            mae_test = sess.run(loss_abs, feed_dict ={x:test_data, y:test_label, keep_prob:1.0,ornot:False, h:H_200_test} )
            cos_test = sess.run(loss_cos, feed_dict ={x:test_data, y:test_label, keep_prob:1.0,ornot:False, h:H_200_test} )
            
            print("step " + str(epoch) + ": loss: " + str(loss_train )+ " rmse_train: " + str( rmse_train )+ \
                "  rmse_test: " + str( rmse_test) +  "   mae_test: " + str(mae_test) +"  Cos: " + str(cos_test))
    coord.request_stop()
    coord.join(threads)  
    
    save_path = saver.save(sess,"network_save\cnn.ckpt")
    print("save to path",save_path)

    end = time.clock()
    print('Running time: %s Seconds'%(end-start))
    
    pre_test = sess.run(prediction, feed_dict ={x:test_data, keep_prob:1.0,ornot:False, h:H_200_test})
    np.savetxt('data\output_altitude_cnn.txt', pre_test)

step 0: loss: 0.98342454 rmse_train: 0.40904972  rmse_test: 0.44670776   mae_test: 0.51202846  Cos: 0.031767607
save to path network_save\linear_cnn.ckpt
Running time: 42.511943 Seconds
